In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install tfa-nightly
!pip install import-ipynb
import import_ipynb
import tensorflow as tf
import numpy as np
import os
import scipy.io as sio
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from tqdm import tqdm
from fastprogress import master_bar, progress_bar
%cd '/content/drive/MyDrive/LMSLoss'

     |████████████████████████████████| 747kB 12.2MB/s 
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=b2bae11f189f535dc9b7d27e8f9b7fa508e9e81586d3fc74fb87620703a9b6f7
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
/content/drive/MyDrive/LMSLoss


In [4]:
%run UnetModel.ipynb
# %run MumfordLoss.ipynb
%run augment_data.ipynb


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 576         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
swish (Swish)                   (None, 128, 128, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [5]:
import keras.backend as K

def dice_coef(y_true, y_pred, smooth=1e-10):
    '''Average dice coefficient per batch.'''
    axes = (1,2,3)
    intersection = K.sum(y_true * y_pred, axis=axes)
    summation = K.sum(y_true + y_pred, axis=axes)
    return K.mean((2.0 * intersection + smooth) / (summation + smooth), axis=0)

def jaccard_coef(y_true, y_pred, smooth=1e-10):
    '''Average jaccard coefficient per batch.'''
    axes = (1,2,3)
    intersection = K.sum(y_true * y_pred, axis=axes)
    union = K.sum(y_true + y_pred, axis=axes) - intersection
    return K.mean( (intersection + smooth) / (union + smooth), axis=0)

In [6]:
x = np.asarray(np.load('./dataACDCA/x_crop_128.npy'),np.float32)
y = np.load('./dataACDCA/y_crop_128.npy')

In [7]:
np.random.seed(123)
len_dev = x.shape[0]//5
len_train = x.shape[0] - len_dev
rand_index = np.random.permutation(x.shape[0])
x_test = x[rand_index[:len_dev]]
y_test = y[rand_index[:len_dev]]
x_train = x[rand_index[len_dev:]]
y_train = y[rand_index[len_dev:]]

In [8]:
# plt.figure(1)
# label_endo = tf.where(y_test == 2.0, 1.0, 0.0)
# label_epi =  tf.where(y_test == 1.0, 1.0, 0.0) + label_endo
# plt.subplot(131),plt.imshow(x[102][...,0],cmap='gray')
# plt.subplot(132),plt.imshow(label_epi[102][...,0],cmap='gray')
# plt.subplot(133),plt.imshow(y_test[102][...,0],cmap='gray')

In [9]:
BATCH_SIZE = 32
buffer_size = x_train.shape[0]

@tf.function
def gen_image(image,mask):
  # image, mask = brightness(image,mask)
  #image, mask = saturation(image,mask)
  image, mask = flip(image,mask)
  image, mask = rotate_image(image,mask)
  image, mask = normalize(image, mask)
  return image, mask
train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(buffer_size).map(gen_image).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test)).map(normalize).batch(BATCH_SIZE*2)

In [10]:
class MSLoss_vs2():
  def levelsetLoss(self,target, output, kernel_size = 7):
    loss = 0.0
    yn = output
    # yn = output[...,:-1]
    # bn = output[...,-1:]
    kernel = tf.ones([kernel_size, kernel_size, 1, yn.shape[-1]],tf.float32) / (kernel_size**2)
    for ich in range(target.shape[-1]):
        target_ = target[...,ich:ich+1]    
        pcentroid_1 = tf.nn.conv2d( target_ * yn, kernel, strides=1, padding='SAME')
        pcentroid_2 =  tf.nn.conv2d(yn+1e-6, kernel, strides=1, padding='SAME')
        # pcentroid = tf.reduce_sum(pcentroid_1, (1,2),keepdims=True)/tf.reduce_sum(pcentroid_2 (1,2),keepdims = True)   
        pcentroid = pcentroid_1 / pcentroid_2
        plevel = target_ - pcentroid
        pLoss = plevel * plevel * yn
        loss += tf.reduce_sum(pLoss)
    return loss
      
  def activeContourLoss(self,y_true,y_pred,outDim =3,smooth=0.001):     
    yTrueOnehot = tf.one_hot(tf.squeeze(tf.cast(y_true,tf.uint8),axis=-1), depth = outDim)
    loss =  y_pred * (1-yTrueOnehot) + (1-y_pred)*yTrueOnehot
    return tf.reduce_mean(loss)
    #this is Luac:
    # loss =  - tf.cast(tf.math.log(1-y_pred+smooth),tf.float32) * (1-yTrueOnehot ) - tf.cast(tf.math.log(y_pred+smooth),tf.float32)*yTrueOnehot
    # return tf.reduce_mean(loss)
                    
  def gradientLoss(self,input,penalty = "l1"):

    dH = tf.math.abs(input[:, 1:, :, :] - input[:, :-1, :, :])
    dW = tf.math.abs(input[:, :, 1:, :] - input[:, :, :-1, :])
    if penalty == "l2":
        dH = dH * dH
        dW = dW * dW

    loss =  tf.reduce_sum(dH) +  tf.reduce_sum(dW)
    return loss

In [11]:
class Mumford_Unet_vs2(MSLoss_vs2):
  def __init__(self,model):
    super().__init__()
    self.model = model
    self.optimizer = None
    self.trainData = None
    self.testData = None
    self.batch_size = BATCH_SIZE
    temp = x_train.shape[0] // self.batch_size
    self.train_step = temp + 1 if x_train.shape[0] % self.batch_size != 0 else temp
    self.learning_rate = 0.001

  def optimizer_fc(self,op_type = "adam", op_param = 0.005):
    if op_type == 'adam':
      self.optimizer =  tf.keras.optimizers.Adam(op_param)
    elif op_type == "nadam":
      self.optimizer =  tf.keras.optimizers.Nadam(op_param)

  def loss(self, image, output, trueLabel, alpha = 1e-3, beta = 0.001):
    # print('\ractive loss: ',self.activeContourLoss(trueLabel, output),' leverset loss: ', self.levelsetLoss(image,output),'gradient loss: ', self.gradientLoss(output), end="")
    return alpha * ( self.levelsetLoss(image,output) + beta * self.gradientLoss(output)) + self.activeContourLoss(trueLabel, output)
    # return self.activeContourLoss(trueLabel, output)

  def metrics(self,y_true,y_pred):
    output_standard = tf.expand_dims(tf.argmax(y_pred,axis=-1),axis = -1)
    output_endo = tf.where(output_standard == 2, 1.0, 0.0)
    output_epi =  tf.where(output_standard == 1, 1.0, 0.0) + output_endo
    label_endo = tf.where(y_true == 2, 1.0, 0.0)
    label_epi =  tf.where(y_true == 1, 1.0, 0.0) + label_endo

    dice_coef_endo = dice_coef(label_endo, output_endo)
    dice_coef_epi = dice_coef(label_epi, output_epi)
    jaccards_coef_endo = jaccard_coef(label_endo, output_endo)
    jaccards_coef_epi = jaccard_coef(label_epi, output_epi)
    return dice_coef_endo, dice_coef_epi, jaccards_coef_endo, jaccards_coef_epi

  def evaluateTest(self,testDataset):
    test_dices_endo =  []
    test_dices_epi = []
    test_jaccards_endo =  []
    test_jaccards_epi = []
    for xBatchTest, yBatchTest in testDataset:
      yPredBatchTest = self.model(xBatchTest, training = False)
      diceEndoTest, diceEpiTest, jaccardEndoTest, jaccardEpiTest =  self.metrics(yBatchTest, yPredBatchTest)

      test_dices_endo.append(diceEndoTest)
      test_dices_epi.append(diceEpiTest)
      test_jaccards_endo.append(jaccardEndoTest)
      test_jaccards_epi.append(jaccardEpiTest)
    return np.mean(test_dices_endo), np.mean(test_dices_epi), np.mean(test_jaccards_endo), np.mean(test_jaccards_epi)
    
  def epoch_training(self,model,optimizer, trainDataset, testDataset, mb, stepTrain, alpha_loss, beta_loss):
    train_losses = []
    train_dices_endo =  []
    train_dices_epi = []
    train_jaccards_endo =  []
    train_jaccards_epi = []

    trainDataset = iter(trainDataset)  
    
    for _ in progress_bar(range(stepTrain),parent = mb):   
      image, y_true = next(trainDataset)
      with tf.GradientTape() as tape:
        y_pred = self.model(image, training = True)
        total_loss  =  self.loss(image, y_pred, y_true, alpha_loss, beta_loss)
      grad  = tape.gradient(total_loss, self.model.trainable_variables)
      optimizer.apply_gradients(zip(grad,self.model.trainable_variables))
      # y_pred, total_loss = self.step(image, y_true)
      diceEndoTrain, diceEpiTrain, jaccardEndoTrain, jaccardEpiTrain =  self.metrics(y_true, y_pred)

      mb.child.comment = 'Train loss {:.4f}'.format(total_loss)

      train_losses.append(total_loss)
      train_dices_endo.append(diceEndoTrain)
      train_dices_epi.append(diceEpiTrain)
      train_jaccards_endo.append(jaccardEndoTrain)
      train_jaccards_epi.append(jaccardEpiTrain)

    trainLossMean = np.mean(train_losses)
    trainDiceEndoMean = np.mean(train_dices_endo)
    trainJaccardEndoMean = np.mean(train_jaccards_endo)
    trainDiceEpiMean = np.mean(train_dices_epi)
    trainJaccardEpiMean = np.mean(train_jaccards_epi)

    testDiceEndoMean, testDiceEpiMean, _, _ = self.evaluateTest(testDataset)

    return trainLossMean, trainDiceEndoMean, trainDiceEpiMean, testDiceEndoMean, testDiceEpiMean


  def train(self,num_epoch, reduceLrEpoch=10 , earlyStoping=22, minLr=1e-5, alpha_loss= 1e-6, beta_loss = 1,checkpoint_prefix = "./weightAlpha(1e-6)/ckpt_{score:.4f}.h5"):
    mb = master_bar(range(num_epoch))
    epochs_list = []
    history = dict()
    training_losses = []
    trainDiceEndo_list = []
    trainDiceEpi_list = []
    testDiceEndo_list = []
    testDiceEpi_list = []
    # learningRate_list = []
    best_score  = 0
    count = 0
    for epoch in mb:
      count += 1
      epochs_list.append(epoch+1)

      trainLoss, trainDiceEndo, trainDiceEpi, testDiceEndo, testDiceEpi  \
      = self.epoch_training(self.model, self.optimizer, self.trainData, self.testData, mb, self.train_step, alpha_loss, beta_loss)

      mb.write('Finish train epoch {} with loss {:.4f} trainDiceEndo: {:.2f}, trainDiceEpi: {:.2f}, testDiceEndo: {:.2f},\
      testDiceEpi: {:.2f}'.format(epoch+1, trainLoss, trainDiceEndo, trainDiceEpi, testDiceEndo, testDiceEpi ))

      training_losses.append(trainLoss)
      trainDiceEndo_list.append(trainDiceEndo)
      trainDiceEpi_list.append(trainDiceEpi)
      testDiceEndo_list.append(testDiceEndo)
      testDiceEpi_list.append(testDiceEpi)
      # learningRate_list.append(self.learning_rate)

      if count % reduceLrEpoch ==0 :
        self.learning_rate = self.learning_rate * 0.5
        if self.learning_rate < minLr:
          self.learning_rate = minLr
        print('learning rate is set to : ',self.learning_rate)
        self.optimizer.learning_rate.assign(self.learning_rate )
    
      mb.update_graph([[epochs_list, training_losses]], [0,num_epoch], [0,0.1])
      if count == earlyStoping :
        break
      ### Check point here ###
      ### Check point follow testDiceEpi ###
      if best_score < testDiceEndo:
          count = 0
          mb.write(">>> Improved Dice-score ENDO from {:.4f} to {:.4f}".format(best_score, testDiceEndo))
          best_score = testDiceEndo
          self.model.save_weights(checkpoint_prefix.format(score=best_score))
    history['epoch'] = epochs_list
    history['train_loss'] = training_losses
    history['train_dice_endo'] = trainDiceEndo_list
    history['train_dice_epi'] = trainDiceEpi_list
    history['test_dice_endo'] = testDiceEndo_list
    history['test_dice_epi'] = testDiceEpi_list
    # history['lr'] = learningRate_list
    return history 

In [12]:
S = seg_net()
nhat = Mumford_Unet_vs2(S)
nhat.learning_rate = 0.001
nhat.trainData = train_dataset
nhat.testData = test_dataset
nhat.optimizer_fc(op_type = "nadam", op_param=nhat.learning_rate)

In [ ]:
# history = nhat.train(200,alpha_loss=1e-7,beta_loss=1e-2,checkpoint_prefix="./weightACDCA_LMS/ckpt_{score:.4f}.h5")

In [ ]:
# sio.savemat('./weightACDCA_LMS/historyACDCA_LMS.mat',history)

In [13]:
nhat.model.load_weights('./weightACDCA_LMS/best_model.h5')

In [14]:
nhat.evaluateTest(test_dataset)

(0.9253606, 0.93679476, 0.8864251, 0.9080615)

In [15]:
for i in range(x_test.shape[0]):
  y_pred = nhat.model(normalize(x_test[i:i+1],y_test[i:i+1]))
  y_predShow = np.argmax(y_pred,axis = -1)
  plt.figure(i+1)
  plt.subplot(131),plt.imshow(x_test[i,...,0],cmap='gray'),plt.title('image')
  plt.subplot(132),plt.imshow(y_predShow[0],cmap ='gray'),plt.title('predict')
  plt.subplot(133),plt.imshow(y_test[i,...,0],cmap='gray'),plt.title('groundtruth')

Output hidden; open in https://colab.research.google.com to view.